# Seq2Seq IA3 for SQL

This is the main notebook for fine-tuning *codet5p-220m-bimodal seq2seq* using $(IA)^3$  PEFT method on SQL programming language dataset.

## Clone the repository

In [1]:
!git clone https://github.com/leiluk1/CodeSearcher.git && cd CodeSearcher/ && git checkout dev/embeddings

Cloning into 'CodeSearcher'...
remote: Enumerating objects: 366, done.
remote: Counting objects: 100% (366/366), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 366 (delta 192), reused 283 (delta 118), pack-reused 0
Receiving objects: 100% (366/366), 31.40 MiB | 27.91 MiB/s, done.
Resolving deltas: 100% (192/192), done.
Branch 'dev/embeddings' set up to track remote branch 'dev/embeddings' from 'origin'.
Switched to a new branch 'dev/embeddings'


## Set up the required dependencies

In [ ]:
!pip install dataprep gdown py7zr transformers peft evaluate rouge_score fire loguru --quiet

In [3]:
# import wandb
# wandb.login(key='')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
!mkdir -p /kaggle/output/CodeSearcher/output
!mkdir -p CodeSearcher/data/raw
!gdown 1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
!unzip -q -d /kaggle/working/CodeSearcher/data/raw ./XLCoST_data.zip

Downloading...
From (uriginal): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
From (redirected): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6&confirm=t&uuid=89396ef0-dea5-44fa-8e4c-62dc837eaf4e
To: /kaggle/working/XLCoST_data.zip
100%|█████████████████████████████████████████| 298M/298M [00:00<00:00, 309MB/s]


## Perform fine-tuning 

For more details, please check the code in `src/models/train.py` in our repository.

In [5]:
!export DATASETS_VERBOSITY=error
!cd CodeSearcher/ && export PYTHONPATH=. && python src/models/train.py seq2seq ia3 \
    --output_dir="output" \
    --epochs=10 \
    --language="SQL" \
    --train_batch_size=16

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-220m-bimodal:
- configuration_codet5p_bimodal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-220m-bimodal:
- modeling_codet5p_bimodal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
2023-11-25 19:01:15.159 | INFO     | __ma

## Save checkpoint

In [6]:
!zip -r seq2seq_ia3_sql.zip CodeSearcher/output

  adding: CodeSearcher/output/ (stored 0%)
  adding: CodeSearcher/output/runs/ (stored 0%)
  adding: CodeSearcher/output/runs/Nov25_19-02-04_f605969cb05d/ (stored 0%)
  adding: CodeSearcher/output/runs/Nov25_19-02-04_f605969cb05d/events.out.tfevents.1700938924.f605969cb05d.180.0 (deflated 63%)
  adding: CodeSearcher/output/checkpoint-4500/ (stored 0%)
  adding: CodeSearcher/output/checkpoint-4500/optimizer.pt (deflated 20%)
  adding: CodeSearcher/output/checkpoint-4500/scheduler.pt (deflated 49%)
  adding: CodeSearcher/output/checkpoint-4500/special_tokens_map.json (deflated 82%)
  adding: CodeSearcher/output/checkpoint-4500/trainer_state.json (deflated 78%)
  adding: CodeSearcher/output/checkpoint-4500/training_args.bin (deflated 49%)
  adding: CodeSearcher/output/checkpoint-4500/vocab.json (deflated 59%)
  adding: CodeSearcher/output/checkpoint-4500/merges.txt (deflated 54%)
  adding: CodeSearcher/output/checkpoint-4500/added_tokens.json (deflated 37%)
  adding: CodeSearcher/output/c

## Evaluation

In this step, test the model using *Mean Reciprocal Rank (MRR)* as the evaluation metric. 

For more details, please refer to `src/models/evaluation.py` in the repository.

In [7]:
!cd CodeSearcher/ && export PYTHONPATH=. && python src/models/evaluation.py \
    --tuned_ckpt_path="output/best_ckpt" \
    --num_virtual_tokens=0 \
    --language="SQL" 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 709.82it/s]
2023-11-26 04:01:19.502 | INFO     | src.datasets.StaQC.make_dataset:_setup_dataset:38 - Loading complete
2023-11-26 04:01:19.594 | INFO     | src.datasets.StaQC.make_dataset:__init__:68 - StaQC SQL train dataset length: 62930
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 646.77it/s]
2023-11-26 04:01:22.269 | INFO     | src.datasets.StaQC.make_dataset:_setup_dataset:38 - Loading complete
2023-11-26 04:01:22.302 | INFO     | src.datasets.StaQC.make_dataset:__init__:68 - StaQC SQL val dataset length: 6993
100%|██████████████

# Results:

- Test MRR for SQL: **0.01844153**;
- Epochs: **10**;
- Trainable params: 82,944 || all params: 223,165,698 || trainable%: 0.03716700225139439.

You can also check the validation results plots via link to [wanb report](https://api.wandb.ai/links/ley-khaertdinova/13lvn64p).
